In [24]:
import pandas as pd
import numpy as np
data = {
    'faculty_id': np.random.choice([100, 200, 300], 50),
    'department': np.random.choice(['Toán', 'Lý', 'Hóa', 'Văn'], 50),
    'student_level': np.random.choice(['Năm 1', 'Năm 2', 'Năm 3', 'Sau Đại học'], 50),
    'lecturer_id': np.random.randint(1, 15, 50),
    'rating_score': np.round(np.random.uniform(3.0, 5.0, 50), 1),
    'comment_sentiment': np.random.choice(['Positive', 'Negative', 'Neutral'], 50, p=[0.4, 0.3, 0.3])
}
df = pd.DataFrame(data)

print("--- DataFrame (5 hàng đầu) ---")
print(df.head())

--- DataFrame (5 hàng đầu) ---
   faculty_id department student_level  lecturer_id  rating_score  \
0         200        Hóa         Năm 1           12           4.1   
1         100        Văn   Sau Đại học            9           3.2   
2         200        Hóa   Sau Đại học            3           3.8   
3         300       Toán   Sau Đại học           12           3.6   
4         100       Toán   Sau Đại học            4           4.4   

  comment_sentiment  
0           Neutral  
1          Negative  
2           Neutral  
3          Positive  
4           Neutral  


1_Tính rating_score trung bình theo faculty_id

In [25]:
avg_rating_by_faculty = df.groupby('faculty_id')['rating_score'].mean().reset_index()
avg_rating_by_faculty.rename(columns={'rating_score': 'avg_rating_faculty'}, inplace=True)

print("\n### 1. Rating trung bình theo Khoa (faculty_id) ###")
print(avg_rating_by_faculty)


### 1. Rating trung bình theo Khoa (faculty_id) ###
   faculty_id  avg_rating_faculty
0         100            3.920000
1         200            3.993750
2         300            3.885714


2_Tính rating_score trung bình theo department

In [26]:
avg_rating_by_dept = df.groupby('department')['rating_score'].mean().reset_index()
avg_rating_by_dept.rename(columns={'rating_score': 'avg_rating_department'}, inplace=True)

print("\n### 2. Rating trung bình theo Bộ môn (department) ###")
print(avg_rating_by_dept.sort_values(by='avg_rating_department', ascending=False))


### 2. Rating trung bình theo Bộ môn (department) ###
  department  avg_rating_department
0        Hóa               3.983333
2       Toán               3.927273
3        Văn               3.917647
1         Lý               3.910000


#Tính rating_score trung bình theo department × student_level

In [27]:
avg_rating_multi_level = df.groupby(['department', 'student_level'])['rating_score'].mean().reset_index()
avg_rating_multi_level.rename(columns={'rating_score': 'avg_rating_dept_level'}, inplace=True)

print("\n### 3. Rating trung bình theo Bộ môn X Cấp độ Sinh viên ###")
# Hiển thị kết quả dưới dạng bảng Pivot (trực quan hơn)
pivot_table = avg_rating_multi_level.pivot(index='department', columns='student_level', values='avg_rating_dept_level')
print(pivot_table)


### 3. Rating trung bình theo Bộ môn X Cấp độ Sinh viên ###
student_level     Năm 1  Năm 2  Năm 3  Sau Đại học
department                                        
Hóa            4.433333  3.625  3.900     4.025000
Lý             4.100000    NaN  3.925     3.860000
Toán           3.966667    NaN  3.800     3.980000
Văn            4.400000  3.800  4.200     3.657143


4_ Tính tỷ lệ bình luận tích cực/tiêu cực theo department

In [28]:
# Lọc chỉ các bình luận Positive và Negative
df_filtered = df[df['comment_sentiment'].isin(['Positive', 'Negative'])]

# GroupBy và đếm số lượng cho từng loại sentiment
sentiment_counts = df_filtered.groupby(['department', 'comment_sentiment']).size().unstack(fill_value=0)

# Tính tổng số bình luận (Positive + Negative)
sentiment_counts['Total_PN'] = sentiment_counts['Positive'] + sentiment_counts['Negative']

# Tính Tỷ lệ Tích cực
sentiment_counts['Positive_Ratio'] = sentiment_counts['Positive'] / sentiment_counts['Total_PN']
sentiment_counts['Negative_Ratio'] = sentiment_counts['Negative'] / sentiment_counts['Total_PN']

print("\n### 4. Tỷ lệ Bình luận Tích cực/Tiêu cực theo Bộ môn ###")
print(sentiment_counts[['Positive', 'Negative', 'Total_PN', 'Positive_Ratio', 'Negative_Ratio']].sort_values(by='Positive_Ratio', ascending=False))


### 4. Tỷ lệ Bình luận Tích cực/Tiêu cực theo Bộ môn ###
comment_sentiment  Positive  Negative  Total_PN  Positive_Ratio  \
department                                                        
Toán                      7         2         9        0.777778   
Văn                       8         5        13        0.615385   
Lý                        4         4         8        0.500000   
Hóa                       3         3         6        0.500000   

comment_sentiment  Negative_Ratio  
department                         
Toán                     0.222222  
Văn                      0.384615  
Lý                       0.500000  
Hóa                      0.500000  


5_Xác định các giảng viên có rating_score trung bình cao nhất/thấp nhất

In [ ]:
import pandas as pd
import numpy as np

faculty = pd.read_csv("data_raw/faculty_info.csv")
survey = pd.read_csv("data_raw/course_survey.csv")
comments = pd.read_csv("data_raw/student_comments.csv")

# Merge survey + faculty
df = survey.merge(faculty, on='faculty_id', how='left')


In [29]:
# Đặt ngưỡng số phiếu khảo sát tối thiểu
MIN_SURVEY_COUNT = 5

# 1. Group và tính cả MEAN và COUNT cùng lúc
lecturer_summary = df.groupby('lecturer_id').agg(
    avg_rating=('rating_score', 'mean'),
    survey_count=('rating_score', 'count')
).reset_index()

# 2. Lọc (Filter) theo điều kiện tối thiểu
qualified_lecturers = lecturer_summary[lecturer_summary['survey_count'] >= MIN_SURVEY_COUNT]

print(f"\n### 5. Giảng viên Cao/Thấp nhất (với MIN_SURVEY_COUNT >= {MIN_SURVEY_COUNT}) ###")

# Giảng viên có Rating Trung bình CAO NHẤT
top_lecturers = qualified_lecturers.sort_values(by='avg_rating', ascending=False).head(5)
print("\n--- TOP 5 Giảng viên có Rating Trung bình CAO NHẤT ---")
print(top_lecturers)

# Giảng viên có Rating Trung bình THẤP NHẤT
bottom_lecturers = qualified_lecturers.sort_values(by='avg_rating', ascending=True).head(5)
print("\n--- TOP 5 Giảng viên có Rating Trung bình THẤP NHẤT ---")
print(bottom_lecturers)


### 5. Giảng viên Cao/Thấp nhất (với MIN_SURVEY_COUNT >= 5) ###

--- TOP 5 Giảng viên có Rating Trung bình CAO NHẤT ---
    lecturer_id  avg_rating  survey_count
10           11    4.183333             6
0             1    4.000000             6
9            10    4.000000             5

--- TOP 5 Giảng viên có Rating Trung bình THẤP NHẤT ---
    lecturer_id  avg_rating  survey_count
0             1    4.000000             6
9            10    4.000000             5
10           11    4.183333             6
